# 학습된 모델 저장

- 학습이 완료된 모델을 파일로 저장하여, 이후 추가 학습이나 예측 서비스에 사용할 수 있도록 한다.
- 파이토치(PyTorch)는 **모델의 파라미터만 저장**하는 방법과 **모델의 구조와 파라미터를 모두 저장**하는 두 가지 방식을 제공한다.
- 저장 함수
  - `torch.save(저장할 객체, 저장 경로)`
- 보통 저장 파일의 확장자는 `.pt`나 `.pth`를 사용한다.

## 모델 전체 저장 및 불러오기

- 저장하기
  - `torch.save(model, 저장 경로)`
- 불러오기
  - `load_model = torch.load(저장 경로)`
- 모델 저장 시 **피클(pickle)**을 사용해 직렬화되므로, 모델을 불러오는 실행 환경에도 저장할 때 사용한 클래스 정의가 필요하다.


## 모델의 파라미터만 저장

-   모델을 구성하는 파라미터만 저장한다.
-   모델의 구조는 저장하지 않기 때문에 불러올 때 **모델을 먼저 생성하고 생성한 모델에 불러온 파라미터를 덮어씌운다.**
-   모델의 파라미터는 **state_dict** 형식으로 저장한다.

### state_dict

-   모델의 파라미터 Tensor들을 레이어 단위별로 나누어 저장한 Ordered Dictionary (OrderedDict)
-   `모델객체.state_dict()` 메소드를 이용해 조회한다.
-   모델의 state_dict을 조회 후 저장한다.
    -   `torch.save(model.state_dict(), "저장경로")`
-   생성된 모델에 읽어온 state_dict를 덮어씌운다.
    -   `new_model.load_state_dict(torch.load("state_dict저장경로"))`


## Checkpoint 저장 및 불러오기

- 학습이 끝나지 않은 모델을 저장하고, 나중에 이어서 학습시키려면 모델의 구조와 파라미터뿐만 아니라 optimizer, loss 함수 등 학습에 필요한 객체들도 함께 저장해야 한다.
- 딕셔너리(Dictionary)에 저장하려는 값들을 key-value 쌍으로 구성하여 `torch.save()`를 이용해 저장한다.

```python
# 저장
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': train_loss
}, "저장경로")

# 불러오기
model = MyModel()
optimizer = optim.Adam(model.parameters())

# 불러온 checkpoint를 이용해 이전 학습 상태 복원
checkpoint = torch.load("저장경로")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
```

In [1]:
# 간단한 모델 정의
import torch
import torch.nn as nn

class MyModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.lr1 = nn.Linear(3, 4) # 3 X 4 + 4 
        self.lr2 = nn.Linear(4, 2)
        self.relu = nn.ReLU() # activation함수->파라미터가 없는 단순 계산함수. relu(X) = max(X, 0)
    def forward(self, X):
        X = self.lr1(X)
        X = self.relu(X)
        X = self.lr2(X)
        return X

In [2]:
# 모델 생성성
model = MyModel()
model

MyModel(
  (lr1): Linear(in_features=3, out_features=4, bias=True)
  (lr2): Linear(in_features=4, out_features=2, bias=True)
  (relu): ReLU()
)

In [3]:
################################################
#  모델에 Layer들을 조회. 모델.instance변수명
################################################
lr_layer = model.lr1
lr_layer

Linear(in_features=3, out_features=4, bias=True)

In [4]:
################################################
#  Layer의 파라미터(weight/bias) 조회
################################################
lr1_weight = lr_layer.weight
lr1_bias = lr_layer.bias

In [5]:
lr1_weight

Parameter containing:
tensor([[-0.0355,  0.3490, -0.1643],
        [-0.2801, -0.2739, -0.5457],
        [-0.4415, -0.0297,  0.3786],
        [ 0.0406,  0.4998, -0.1034]], requires_grad=True)

In [6]:
lr1_bias

Parameter containing:
tensor([ 0.3467, -0.0845, -0.4996, -0.0269], requires_grad=True)

In [7]:
import os
os.makedirs("saved_models", exist_ok=True)

In [8]:
################################################
#  모델을 저장
################################################
torch.save(model, "saved_models/my_model.pt")

In [9]:
################################################
#  저장된 모델 Load
################################################
load_model = torch.load("saved_models/my_model.pt", weights_only=False)

In [10]:
load_model

MyModel(
  (lr1): Linear(in_features=3, out_features=4, bias=True)
  (lr2): Linear(in_features=4, out_features=2, bias=True)
  (relu): ReLU()
)

In [11]:
load_model.lr1.weight

Parameter containing:
tensor([[-0.0355,  0.3490, -0.1643],
        [-0.2801, -0.2739, -0.5457],
        [-0.4415, -0.0297,  0.3786],
        [ 0.0406,  0.4998, -0.1034]], requires_grad=True)

In [12]:
lr1_weight

Parameter containing:
tensor([[-0.0355,  0.3490, -0.1643],
        [-0.2801, -0.2739, -0.5457],
        [-0.4415, -0.0297,  0.3786],
        [ 0.0406,  0.4998, -0.1034]], requires_grad=True)

In [13]:
######################################################
# 모델의 파라미터들(weight들, bias들)만 저장/불러오기
######################################################
state_dict = model.state_dict()
state_dict

OrderedDict([('lr1.weight',
              tensor([[-0.0355,  0.3490, -0.1643],
                      [-0.2801, -0.2739, -0.5457],
                      [-0.4415, -0.0297,  0.3786],
                      [ 0.0406,  0.4998, -0.1034]])),
             ('lr1.bias', tensor([ 0.3467, -0.0845, -0.4996, -0.0269])),
             ('lr2.weight',
              tensor([[-0.3335, -0.3852, -0.3446,  0.2252],
                      [ 0.3588,  0.4776, -0.2790,  0.1461]])),
             ('lr2.bias', tensor([-0.4323, -0.2951]))])

In [14]:
state_dict.keys()

odict_keys(['lr1.weight', 'lr1.bias', 'lr2.weight', 'lr2.bias'])

In [15]:
###################
# state_dict 저장
################### 

torch.save(state_dict, "saved_models/my_model_parameter.pt")

In [16]:
#####################
# state_dict load
#####################
sd = torch.load("saved_models/my_model_parameter.pt")

In [17]:
# load한 state_dict를 모델 파라미터에 적용(덮어 씌운다.)
new_model = MyModel()
new_model.state_dict()

OrderedDict([('lr1.weight',
              tensor([[-0.4270,  0.0882, -0.5171],
                      [ 0.1044, -0.5147, -0.2236],
                      [ 0.1398, -0.1062,  0.3185],
                      [-0.2263, -0.5404, -0.1163]])),
             ('lr1.bias', tensor([ 0.4479, -0.4328,  0.4422, -0.3472])),
             ('lr2.weight',
              tensor([[-0.2066, -0.4670,  0.4441, -0.4921],
                      [ 0.0881, -0.4068, -0.2147,  0.4300]])),
             ('lr2.bias', tensor([ 0.2005, -0.0150]))])

In [18]:
new_model.load_state_dict(sd)

<All keys matched successfully>

In [19]:
new_model.state_dict()

OrderedDict([('lr1.weight',
              tensor([[-0.0355,  0.3490, -0.1643],
                      [-0.2801, -0.2739, -0.5457],
                      [-0.4415, -0.0297,  0.3786],
                      [ 0.0406,  0.4998, -0.1034]])),
             ('lr1.bias', tensor([ 0.3467, -0.0845, -0.4996, -0.0269])),
             ('lr2.weight',
              tensor([[-0.3335, -0.3852, -0.3446,  0.2252],
                      [ 0.3588,  0.4776, -0.2790,  0.1461]])),
             ('lr2.bias', tensor([-0.4323, -0.2951]))])

In [20]:
print(model)

MyModel(
  (lr1): Linear(in_features=3, out_features=4, bias=True)
  (lr2): Linear(in_features=4, out_features=2, bias=True)
  (relu): ReLU()
)


In [21]:
# torchinfo 패키지 설치: 파이토치 모델 구조를 조사해주는 패키지.
# !pip install torchinfo

Resolved 1 package in 71ms
Prepared 1 package in 34ms
Installed 1 package in 19ms
 + torchinfo==1.8.0


In [21]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                   Param #
MyModel                                  --
├─Linear: 1-1                            16
├─Linear: 1-2                            10
├─ReLU: 1-3                              --
Total params: 26
Trainable params: 26
Non-trainable params: 0

In [22]:
# input data 의 shape을 지정하면 각 Layer의 output shape을 출력한다.
summary(model, (100, 3))

Layer (type:depth-idx)                   Output Shape              Param #
MyModel                                  [100, 2]                  --
├─Linear: 1-1                            [100, 4]                  16
├─ReLU: 1-2                              [100, 4]                  --
├─Linear: 1-3                            [100, 2]                  10
Total params: 26
Trainable params: 26
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

# 문제 유형별 MLP 네트워크
- 해결하려는 문제 유형에 따라 출력 Layer의 구조가 바뀐다.
- 딥러닝 구조에서 **Feature를 추출하는 Layer 들을 Backbone** 이라고 하고 **추론하는 Layer들을 Head** 라고 한다. 


> - MLP(Multi Layer Perceptron), DNN(Deep Neural Network), ANN(Artificial Neural Network)
>     -   Fully Connected Layer(nn.Linear)로 구성된 딥러닝 모델
>     -   input feature들 모두에 대응하는 weight들(가중치)을 사용한다.
> 

## Boston Housing Dataset - **Regression(회귀) 문제**

보스턴 주택가격 dataset은 다음과 같은 속성을 바탕으로 해당 타운 주택 가격의 중앙값을 예측하는 문제.

-   CRIM: 범죄율
-   ZN: 25,000 평방피트당 주거지역 비율
-   INDUS: 비소매 상업지구 비율
-   CHAS: 찰스강에 인접해 있는지 여부(인접:1, 아니면:0)
-   NOX: 일산화질소 농도(단위: 0.1ppm)
-   RM: 주택당 방의 수
-   AGE: 1940년 이전에 건설된 주택의 비율
-   DIS: 5개의 보스턴 직업고용센터와의 거리(가중 평균)
-   RAD: 고속도로 접근성
-   TAX: 재산세율
-   PTRATIO: 학생/교사 비율
-   B: 흑인 비율
-   LSTAT: 하위 계층 비율
    <br><br>
-   **Target**
    -   MEDV: 타운의 주택가격 중앙값(단위: 1,000달러)


In [23]:
import pandas as pd

df = pd.read_csv('data/boston_housing.csv')
print(df.shape)

(506, 14)


In [24]:
X = df.drop(columns='MEDV').values
y = df['MEDV'].values
X.shape , y.shape

((506, 13), (506,))

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=42)

In [26]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
# Train/Test Dataset
from torch.utils.data import TensorDataset
trainset = TensorDataset(
    torch.tensor(X_train,dtype=torch.float32),
    torch.tensor(y_train,dtype=torch.float32)
)

testset = TensorDataset(
    torch.tensor(X_test,dtype=torch.float32),
    torch.tensor(y_test,dtype=torch.float32)
)

len(trainset) , len(testset)    

(404, 102)

In [28]:
# DataLoader
from torch.utils.data import DataLoader
batch_size = 200
train_loader = DataLoader(trainset,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader = DataLoader(testset,batch_size=len(testset))

In [29]:
##################
# 모델 정의
#################

import torch
import torch.nn as nn
class BostonModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.lr1 = nn.Linear(13,32) # 첫번째 연산
        self.lr2 = nn.Linear(32,16)
        self.lr3 = nn.Linear(16,1)    # 추론 결과를 출력
        self.relu = nn.ReLU()

    def forward(self,X):
        # Linear - > ReLU -> Linear - > ReLU -> Linear -> 출력 
        out = self.lr1(X)
        out = self.relu(out)
        out = self.lr2(out)
        out = self.relu(out)
        out = self.lr3(out)
        return out

In [30]:
epochs = 1000
lr = 0.001
device = 'cuda' if torch.cuda.is_available () else 'cpu'

train_loss_list = []
valid_loss_list = []

# 모델 생성
boston_model = BostonModel().to(device)
# Loss 함수
loss_fn = nn.MSELoss()
# Optimizer
optimizer = torch.optim.RMSprop(boston_model.parameters(),lr =lr)

In [31]:
from torchinfo import summary
summary(boston_model,(100,13),device=device)

Layer (type:depth-idx)                   Output Shape              Param #
BostonModel                              [100, 1]                  --
├─Linear: 1-1                            [100, 32]                 448
├─ReLU: 1-2                              [100, 32]                 --
├─Linear: 1-3                            [100, 16]                 528
├─ReLU: 1-4                              [100, 16]                 --
├─Linear: 1-5                            [100, 1]                  17
Total params: 993
Trainable params: 993
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.10
Input size (MB): 0.01
Forward/backward pass size (MB): 0.04
Params size (MB): 0.00
Estimated Total Size (MB): 0.05

In [32]:
# Train - > Train / Validation 두 단계
for epoch in range(epochs):
    ##########################
    # 학습(train)
    ##########################
    boston_model.train()
    train_loss = 0.0 # 현 epoch의 loss값을 저장할 변수
    for X_train, y_train in train_loader: # 1 배치 학습
        # 1. X,y 를 device 로 이동. (model 과 같은 device 로 이동)
        X_train, y_train = X_train.to(device) , y_train.to(device)

        # 2. 추론
        pred_train = boston_model(X_train)

        # 3. Loss 계산
        loss = loss_fn(pred_train,y_train)

        # 4.역전파로 gradient 계산
        loss.backward()

        # 5. 파라미터 업데이트
        optimizer.step()

        # 6. gradient 초기화
        optimizer.zero_grad()

        # 7. 현재 batch에 대한 loss 계산
        train_loss += loss.item()
    
    train_loss_list.append(train_loss/len(train_loader))

    ##########################
    # 검증(validation)
    ##########################
    model.eval()

    valid_loss = 0.0 # 현 epoch에 대한 검증 loss값을 저장할 변수
    with torch.no_grad():
        for X_valid,y_valid in test_loader:
            # 1. X,y 를 model의 device 로 이동.
            X_valid,y_valid = X_valid.to(device),y_valid.to(device)

            # 2. 추론
            pred_valid = boston_model(X_valid)

            #3. 평가 - MSE
            valid_loss = loss_fn(pred_valid,y_valid).item()
        valid_loss /= len(test_loader)
        valid_loss_list.append(valid_loss)
    # 로그 출력 train/valid loss를 출력
    if epoch % 100 == 0 or epoch == epochs-1:
        print(f'[{epoch+1}/{epoch}] train_loss : {train_loss:.5f}, validation_loss : {valid_loss:.5f}')

c:\Users\Playdata\Documents\SKN21_Yoo\07_deeplearning_pytorch\.venv\Lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Playdata\Documents\SKN21_Yoo\07_deeplearning_pytorch\.venv\Lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([102])) that is different to the input size (torch.Size([102, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1/0] train_loss : 1190.78381, validation_loss : 515.63776
[101/100] train_loss : 193.13811, validation_loss : 83.59399
[201/200] train_loss : 181.27630, validation_loss : 78.34252
[301/300] train_loss : 178.65058, validation_loss : 77.07653
[401/400] train_loss : 176.42942, validation_loss : 76.56023
[501/500] train_loss : 174.34029, validation_loss : 76.37154
[601/600] train_loss : 176.33964, validation_loss : 76.18123
[701/700] train_loss : 176.52665, validation_loss : 76.24580
[801/800] train_loss : 174.81928, validation_loss : 76.16862
[901/900] train_loss : 175.20389, validation_loss : 75.39146
[1000/999] train_loss : 175.37435, validation_loss : 75.59916


In [33]:
save_boston_model_path = 'saved_models/boston_model.pth'
torch.save(boston_model,save_boston_model_path)
torch.save(boston_model.state_dict(),'saved_models/boston_model_parameter.pth')

In [34]:
load_model = torch.load(save_boston_model_path,weights_only=False)
load_model.eval()
with torch.no_grad():
    print(load_model(X_valid)[:10])

tensor([[23.2140],
        [22.6164],
        [23.5124],
        [23.9293],
        [22.8345],
        [23.0422],
        [22.1576],
        [22.2386],
        [23.2756],
        [22.5179]])


In [35]:

load_state_dict = torch.load('saved_models/boston_model_parameter.pth',weights_only = False)
new_model = BostonModel()
new_model.load_state_dict(load_state_dict)
new_model.eval()
with torch.no_grad():
    result = new_model(X_valid)[:10]
    print(result)


tensor([[23.2140],
        [22.6164],
        [23.5124],
        [23.9293],
        [22.8345],
        [23.0422],
        [22.1576],
        [22.2386],
        [23.2756],
        [22.5179]])


## 분류 (Classification)

### Fashion MNIST Dataset - **다중분류(Multi-Class Classification) 문제**

10개의 범주(category)와 70,000개의 흑백 이미지로 구성된 [패션 MNIST](https://github.com/zalandoresearch/fashion-mnist) 데이터셋.
이미지는 해상도(28x28 픽셀)가 낮고 다음처럼 개별 의류 품목을 나타낸다:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>그림</b> <a href="https://github.com/zalandoresearch/fashion-mnist">패션-MNIST 샘플</a> (Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

- **Feature**이미지는 28x28 크기이며 Gray scale이다.
- **Target**은 총 10개의 class로 구성되어 있으며 각 class의 class 이름은 다음과 같다.

| 레이블 | 클래스       |
|--------|--------------|
| 0      | T-shirt/top |
| 1      | Trousers    |
| 2      | Pullover    |
| 3      | Dress       |
| 4      | Coat        |
| 5      | Sandal      |
| 6      | Shirt       |
| 7      | Sneaker     |
| 8      | Bag         |
| 9      | Ankle boot  |

> #### 학습 도중 모델 저장
>
> - 학습 도중 가장 좋은 성능을 보이는 모델이 나올 수 있다.
> - 학습 도중 모델을 저장하는 방법
>   1. 각 에폭이 끝날 때 마다 모델을 저장한다.
>   2. 한 에폭 학습 후 성능 개선이 있으면 모델을 저장하여 가장 성능 좋은 모델만 저장되도록 한다.
>      - 최고 성능 점수(best score)와 현재 에폭의 성능을 비교하여, 성능이 개선되었을 경우 모델을 저장(덮어쓰기)한다.
>
> #### 조기 종료(Early Stopping)
>
> - 학습 도중 성능 개선이 나타나지 않으면, 중간에 학습을 종료하도록 구현한다.
> - 에폭 수를 충분히 길게 설정한 뒤, 특정 횟수 동안 성능 개선이 없으면 학습을 조기 종료하도록 구현한다.


In [36]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader , random_split
from torchvision.datasets import FashionMNIST
from torchvision import transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'mps' if torch.mps.is_available() else 'cpu'

device

'cpu'

In [37]:
#Dataset
dataset_path = 'dataset/fashion_mnist'
batch_size=256
f_trainset = FashionMNIST(dataset_path,train=True,download=True,transform=transforms.ToTensor())
f_testset = FashionMNIST(dataset_path,train=False,download=True,transform=transforms.ToTensor())
f_trainset, f_validset = random_split(f_trainset,[50000,10000])

f_train_loader = DataLoader(f_trainset,batch_size=batch_size,shuffle=True,drop_last=True)
f_valid_loader = DataLoader(f_validset,batch_size=batch_size)
f_test_loader = DataLoader(f_testset,batch_size=batch_size)

len(f_trainset) , len(f_testset) ,len(f_validset) , len(f_train_loader) , len(f_test_loader)

(50000, 10000, 10000, 195, 40)

In [38]:
f_testset.class_to_idx

{'T-shirt/top': 0,
 'Trouser': 1,
 'Pullover': 2,
 'Dress': 3,
 'Coat': 4,
 'Sandal': 5,
 'Shirt': 6,
 'Sneaker': 7,
 'Bag': 8,
 'Ankle boot': 9}

In [39]:
f_testset.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [40]:
print(f_trainset[0][0].dtype)
print(f_trainset[0][0].size())
print(f_trainset[0][0].min(),f_trainset[0][0].max())

torch.float32
torch.Size([1, 28, 28])
tensor(0.) tensor(1.)


In [41]:
##################
# 모델 정의
#################

class FashionMNISTModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.lr1=nn.Linear(784,2048)
        self.lr2=nn.Linear(2048,1024)
        self.lr3=nn.Linear(1024,512)
        self.lr4=nn.Linear(512,256)
        self.lr5=nn.Linear(256,128)
        self.lr6=nn.Linear(128,64)
        self.lr7=nn.Linear(64,10)
        # 출력 Layer out_features 개수 : 다중분류문제의 경우 정답 클래스 개수.
        self.relu = nn.ReLU()


    def forward(self,X):
        # X shape: (batch,1,28,28) -> (batch,784) torch.flatten(start_dim=1)함수
        out = nn.Flatten()(X) # 0축은 그대로 두고 그 이후 축을 flatten 한다.
        out = self.lr1(out) # self.relu(self.lr1(out))
        out = self.relu(out)

        out = self.lr2(out) 
        out = self.relu(out)
        
        out = self.lr3(out) 
        out = self.relu(out)
        
        out = self.lr4(out) 
        out = self.relu(out)
        
        out = self.lr5(out) 
        out = self.relu(out)
        
        out = self.lr6(out) 
        out = self.relu(out)
        
        out = self.lr7(out)
        return out 

In [42]:
from torchinfo import summary
summary(FashionMNISTModel(),(100,1,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
FashionMNISTModel                        [100, 10]                 --
├─Linear: 1-1                            [100, 2048]               1,607,680
├─ReLU: 1-2                              [100, 2048]               --
├─Linear: 1-3                            [100, 1024]               2,098,176
├─ReLU: 1-4                              [100, 1024]               --
├─Linear: 1-5                            [100, 512]                524,800
├─ReLU: 1-6                              [100, 512]                --
├─Linear: 1-7                            [100, 256]                131,328
├─ReLU: 1-8                              [100, 256]                --
├─Linear: 1-9                            [100, 128]                32,896
├─ReLU: 1-10                             [100, 128]                --
├─Linear: 1-11                           [100, 64]                 8,256
├─ReLU: 1-12                             [100, 64]    

In [43]:
f_model = FashionMNISTModel().to(device)
optimizer = torch.optim.Adam(f_model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss() 

# 다중분류의 loss : CrossEntropyLoss
#  CrossEntropyLoss()입력 : 정답-class index 로 구성 [[3],[8],[0]...], 모델 추정결과 : Softmax를 적용하기 전값 = logit

In [54]:
import time
train_loss_list = []
valid_loss_list = []
valid_acc_list = []

######################
# 1. 성능이 개선될 때 마다 모델을 저장.
# 최종적으로 가장 성능이 좋은 epoch의 모델이 저장되도록
# 2. 일정 epoch 동산 성능개선이 안되면 학습을 중단.
# 이것과 관련된 변수들 정의
######################

best_score = torch.inf #가장 좋은 valid_loss 값을 저장
# 저장
save_model_path = 'saved_models/fashion_mnist_model.pth'
# 조기종료
patience = 10 # 성능이 개선되는지 10 에폭 동안 기다려본다.
stop_count = 0 # 학습하는 도중 몇 번동안 성능개선이 안되었는지 저장할변수. stop_count == patience 이면 종료.

epochs = 100

s = time.time()
for epoch in range(epochs):
    # 학습 
    f_model.train()

    train_loss = 0.0
    for X_train, y_train in f_train_loader:
        # 1. X,y를 device 로 이동
        X_train, y_train = X_train.to(device) , y_train.to(device)
        # 2. 모델 추론
        pred_train = f_model(X_train)
        # 3. loss 계산
        loss = loss_fn(pred_train,y_train)
        # 4. gradient 계산 (역전파)
        loss.backward()
        # 5. 파라미터 업데이트
        optimizer.step()
        # 6. 파라미터 초기화
        optimizer.zero_grad()
        # 7. 현제 에폭에서의 train loss 누적
        train_loss += loss.item()
    
    #train loss 평균계산
    train_loss /=len(f_train_loader)
    train_loss_list.append(train_loss)
    
    # 검증
    f_model.eval()
    valid_loss = 0.0
    valid_acc = 0.0

    with torch.no_grad():
        for X_valid,y_valid in f_valid_loader:
            # 1. device 이동
            X_valid,y_valid = X_valid.to(device),y_valid.to(device)

            # 2. 모델 추정
            pred_valid = f_model(X_valid) # 10개 class 에 대한 logit
            pred_label = pred_valid.argmax(dim=1) # 가장 확률높은 class

            valid_loss += loss_fn(pred_valid,y_valid).item() #현 스텝에서의 loss
            valid_acc += torch.sum(pred_label == y_valid).item() #현 스텝에서 맞은 것 개수.

        valid_loss /= len(f_valid_loader)
        valid_acc /= len(f_validset)
    
        valid_loss_list.append(valid_loss)
        valid_acc_list.append(valid_acc)
        
        #로그 출력 - train_loss , valid_loss , valid_acc
        print(f'[{epoch+1}/{epochs}] train_loss : {train_loss:.5f}, valid_loss : {valid_loss:.5f}, valid_acc : {valid_acc:.5f}')

        #########################
        # 모델 저장 , 조기 종료
        #########################

        if valid_loss < best_score: # 이번 epoch 학습엣 성능(valid_loss)이 개선되었는지
            # 모델 저장
            torch.save(f_model,save_model_path)
            print(f'>>>>>>> 모델 저장 : {epoch+1} epoch - 이전 score : {best_score} , 개선된 score : {valid_loss}')
            best_score = valid_loss
            # 조기 종료 (stop_count를 초기화)
            stop_count = 0
        else: # 성능 개선이 안됨. -> 조기 종료
            stop_count +=1
            if stop_count == patience : 
                print (f'>>>>>>>>>>>> 조기종료 : {best_score} 보다 개선되지 않음.')
                break
            

e = time.time()


print('학습에 걸린 시간 : ',(e-s),' 초')

[1/100] train_loss : 1.15894, valid_loss : 0.73548, valid_acc : 0.73590
>>>>>>> 모델 저장 : 1 epoch - 이전 score : inf , 개선된 score : 0.7354829505085945
[2/100] train_loss : 0.62372, valid_loss : 0.58376, valid_acc : 0.78920
>>>>>>> 모델 저장 : 2 epoch - 이전 score : 0.7354829505085945 , 개선된 score : 0.5837574489414692
[3/100] train_loss : 0.53521, valid_loss : 0.56680, valid_acc : 0.79740
>>>>>>> 모델 저장 : 3 epoch - 이전 score : 0.5837574489414692 , 개선된 score : 0.5668026894330979
[4/100] train_loss : 0.49426, valid_loss : 0.48880, valid_acc : 0.82850
>>>>>>> 모델 저장 : 4 epoch - 이전 score : 0.5668026894330979 , 개선된 score : 0.4888020746409893
[5/100] train_loss : 0.46103, valid_loss : 0.45876, valid_acc : 0.83980
>>>>>>> 모델 저장 : 5 epoch - 이전 score : 0.4888020746409893 , 개선된 score : 0.45876314416527747
[6/100] train_loss : 0.42642, valid_loss : 0.43288, valid_acc : 0.84780
>>>>>>> 모델 저장 : 6 epoch - 이전 score : 0.45876314416527747 , 개선된 score : 0.43287878334522245
[7/100] train_loss : 0.40699, valid_loss : 0.4

In [ ]:
import pickle
result_dict = {
    'train_loss': train_loss_list,
    'valid_loss': valid_loss_list,
    'valid_acc': valid_acc_list
}

with open('f_mnist_train_result.pkl','wb') as fo:
    pickle.dump(result_dict, fo)

### 위스콘신 유방암 데이터셋 - **이진분류(Binary Classification) 문제**

-   **이진 분류 문제 처리 모델의 두가지 방법**
    1. positive(1)일 확률을 출력하도록 구현
        - output layer: units=1, activation='sigmoid'
        - loss: binary_crossentropy
    2. negative(0)일 확률과 positive(1)일 확률을 출력하도록 구현 => 다중분류 처리 방식으로 해결
        - output layer: units=2, activation='softmax', y(정답)은 one hot encoding 처리
        - loss: categorical_crossentropy
-   위스콘신 대학교에서 제공한 종양의 악성/양성여부 분류를 위한 데이터셋
-   Feature
    -   종양에 대한 다양한 측정값들
-   Target의 class
    -   0 - malignant(악성종양)
    -   1 - benign(양성종양)


In [4]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [5]:
# Dataset
X, y = load_breast_cancer(return_X_y=True)
y = y.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=0)

In [6]:
# 전처리
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# Dataset
## 모델의 weight, bias -> float32. X, y는 weight, bias와 계산을 하게 되기 때문에 타입을 맞춰준다.
trainset = TensorDataset(
    torch.tensor(X_train_scaled, dtype=torch.float32),  
    torch.tensor(y_train, dtype=torch.float32)
)
testset = TensorDataset(
    torch.tensor(X_test_scaled, dtype=torch.float32), 
    torch.tensor(y_test, dtype=torch.float32)
)

In [8]:
# class name <-> class index
classes = np.array(["악성종양", "양성종양"])
class_to_idx = {"악성종양":0, "양성종양":1}

trainset.classes = classes
trainset.class_to_idx = class_to_idx

In [9]:
# DataLoader
train_loader = DataLoader(trainset, batch_size=200, shuffle=True, drop_last=True)
test_loader = DataLoader(testset, batch_size=100)

In [11]:
######### 모델 정의
class BCModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lr1=nn.Linear(30,16)
        self.lr2=nn.Linear(16,8)
        self.lr3=nn.Linear(8,1) # out_features : 1 - positive일 확률
        self.relu = nn.ReLU() # hidden layer의 활성함수
        self.sigmoid = nn.Sigmoid() # Linear 출력값을 0 ~ 1 확률로 만들어 주는 Sigmoid(Logistic) 함수.

    def forward(self,X):
        out= self.relu(self.lr1(X))
        out= self.relu(self.lr2(out))
        
        out = self.lr3(out)
        out = self.sigmoid(out)
        return out

In [12]:
m = BCModel()
p = m(torch.randn(10,30))
p

tensor([[0.5971],
        [0.5899],
        [0.5825],
        [0.5649],
        [0.5819],
        [0.5731],
        [0.5618],
        [0.5826],
        [0.5648],
        [0.5586]], grad_fn=<SigmoidBackward0>)

In [13]:
######### 학습
train_loss_list = []
valid_loss_list = []
valid_acc_list = []

epochs = 1000

model = BCModel().to(device)
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

#조기종료 , best 모델 지정
best_score = torch.inf  # valid loss 기준
save_model_path = "saved_models/bc_model.pth"
patience = 10
stop_count = 0

In [14]:
######### 저장된 모델 로드
for epoch in range(epochs):
    # 학습
    model.train()
    train_loss = 0.0
    for X_train,y_train in train_loader:
        # 1. X,y를 device 로 이동
        X_train, y_train = X_train.to(device) , y_train.to(device)
        # 2. 모델 추론
        pred_train = model(X_train)
        # 3. loss 계산
        loss = loss_fn(pred_train,y_train)
        # 4. gradient 계산 (역전파)
        loss.backward()
        # 5. 파라미터 업데이트
        optimizer.step()
        # 6. 파라미터 초기화
        optimizer.zero_grad()
        # 7. loss 누적
        train_loss += loss.item()
    #train loss 평균
    train_loss /= len(train_loader)
    train_loss_list.append(train_loss)

    # 검증
    model.eval()
    valid_loss = 0
    valid_acc = 0
    with torch.no_grad():
        for X_valid,y_valid in test_loader:
            # 1. device 이동
            X_valid,y_valid = X_valid.to(device),y_valid.to(device)

            # 2. 추론
            pred_valid = model(X_valid) # (batch,1) 1축 : positve일 확률
            pred_label = (pred_valid>0.5).type(torch.int32) # label

            # 평가
            valid_loss += loss_fn(pred_valid,y_valid).item() # loss
            valid_acc += torch.sum(pred_label == y_valid).item() # acc

        valid_loss /= len(test_loader)
        valid_acc /= len(testset)
    
        valid_loss_list.append(valid_loss)
        valid_acc_list.append(valid_acc)

        print(f'[{epoch+1}/{epochs}] train_loss : {train_loss:.5f}, valid_loss : {valid_loss:.5f}, valid_acc : {valid_acc:.5f}')

        # 성능 개선시 모델 저장  + 조기종료
        if valid_loss < best_score : #개선됨
            #모델 저장 + stop_count 초기화
            print(f'>>>>>>>>>> {epoch+1}모델을 저장합니다. {best_score} 에서 {valid_loss} 로 개선됨')
            torch.save(model, save_model_path)
            best_score = valid_loss
            stop_count = 0
        else: # 개선 안됨
            stop_count += 1
            if patience == stop_count:
                print(f'>>>>>>>>>> {epoch+1}에서 조기종료 합니다. {best_score} 에서 개선이 안됨')
                break
        

[1/1000] train_loss : 0.69035, valid_loss : 0.66595, valid_acc : 0.86713
>>>>>>>>>> 1모델을 저장합니다. inf 에서 0.6659479737281799 로 개선됨
[2/1000] train_loss : 0.65690, valid_loss : 0.63202, valid_acc : 0.88811
>>>>>>>>>> 2모델을 저장합니다. 0.6659479737281799 에서 0.6320174634456635 로 개선됨
[3/1000] train_loss : 0.61326, valid_loss : 0.59132, valid_acc : 0.90210
>>>>>>>>>> 3모델을 저장합니다. 0.6320174634456635 에서 0.5913179814815521 로 개선됨
[4/1000] train_loss : 0.56640, valid_loss : 0.54829, valid_acc : 0.90210
>>>>>>>>>> 4모델을 저장합니다. 0.5913179814815521 에서 0.5482855141162872 로 개선됨
[5/1000] train_loss : 0.52146, valid_loss : 0.50936, valid_acc : 0.90210
>>>>>>>>>> 5모델을 저장합니다. 0.5482855141162872 에서 0.5093619227409363 로 개선됨
[6/1000] train_loss : 0.48355, valid_loss : 0.47845, valid_acc : 0.93007
>>>>>>>>>> 6모델을 저장합니다. 0.5093619227409363 에서 0.4784467965364456 로 개선됨
[7/1000] train_loss : 0.45267, valid_loss : 0.45644, valid_acc : 0.92308
>>>>>>>>>> 7모델을 저장합니다. 0.4784467965364456 에서 0.45643986761569977 로 개선됨
[8/1000] trai

In [15]:
#저장된 모델 로드
load_bc_model = torch.load(save_model_path, weights_only=False)
load_bc_model

BCModel(
  (lr1): Linear(in_features=30, out_features=16, bias=True)
  (lr2): Linear(in_features=16, out_features=8, bias=True)
  (lr3): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [16]:
# bool type label ex
p = m(torch.randn(10,30))
(p>0.5).type(torch.int32) # bool -> int : True -1 ,False -0

tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]], dtype=torch.int32)

In [17]:
###### 추론 함수 #######
def predict_bc(model, X, device="cpu"):
    # model로 X를 추론한 결과를 반환
    # label, 확률
    result = []
    with torch.no_grad():
        pred_proba = model(X)
        pred_class = (pred_proba > 0.5).type(torch.int32)
        for class_index, proba in zip(pred_class, pred_proba):
            # print(class_index, proba if class_index.item() == 1 else 1-proba)
            result.append((class_index.item(), proba.item() if class_index.item() == 1 else (1-proba.item())))
        return result

In [18]:
new_data = torch.tensor(X_test_scaled[:5], dtype=torch.float32)

In [19]:
result = predict_bc(load_bc_model,new_data)
result

[(1, 0.9975098371505737),
 (0, 0.9999999956312995),
 (0, 0.9999993686446942),
 (1, 0.9999111890792847),
 (0, 0.9999732061987743)]

In [20]:
model = BCModel().to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [21]:
from module.train import fit

result = fit(
    train_loader,test_loader,model,loss_fn,optimizer,epochs=1000,
    save_best_model = True,save_model_path ='saved_models/bc_mode12.pth',
    device=device , mode='binary' 
)

Epoch[1/1000] - Train loss: 0.67202 Train Accucracy: 0.59624 || Validation Loss: 0.66895 Validation Accuracy: 0.64336
저장: 1 - 이전 : inf, 현재: 0.6689500212669373
Epoch[2/1000] - Train loss: 0.66406 Train Accucracy: 0.63146 || Validation Loss: 0.66225 Validation Accuracy: 0.66434
저장: 2 - 이전 : 0.6689500212669373, 현재: 0.6622499525547028
Epoch[3/1000] - Train loss: 0.65799 Train Accucracy: 0.65728 || Validation Loss: 0.65555 Validation Accuracy: 0.69231
저장: 3 - 이전 : 0.6622499525547028, 현재: 0.6555511951446533
Epoch[4/1000] - Train loss: 0.64912 Train Accucracy: 0.70188 || Validation Loss: 0.64892 Validation Accuracy: 0.72028
저장: 4 - 이전 : 0.6555511951446533, 현재: 0.6489169895648956
Epoch[5/1000] - Train loss: 0.64296 Train Accucracy: 0.72066 || Validation Loss: 0.64231 Validation Accuracy: 0.74825
저장: 5 - 이전 : 0.6489169895648956, 현재: 0.6423135697841644
Epoch[6/1000] - Train loss: 0.63701 Train Accucracy: 0.75117 || Validation Loss: 0.63582 Validation Accuracy: 0.78322
저장: 6 - 이전 : 0.642313569784

# 모델 유형별 구현 정리

## 공통

-   Input layer(첫번째 Layer)의 in_features
    -   입력데이터의 feature(속성) 개수에 맞춰준다.
-   Hidden layer 수
    -   경험적(art)으로 정한다.
    -   Hidden layer에 Linear를 사용하는 경우 보통 feature 수를 줄여 나간다. (핵심 특성들을 추출해나가는 과정의 개념.)

## 회귀 모델

-   output layer의 출력 unit개수(out_features)
    -   정답의 개수
    -   ex
        -   집값: 1
        -   아파트가격, 단독가격, 빌라가격: 3 => y의 개수에 맞춘다.
-   출력 Layer에 적용하는 activation 함수
    -   일반적으로 **None**
    -   값의 범위가 설정되 있고 그 범위의 값을 출력하는 함수가 있을 경우
        -   ex) 0 ~ 1: logistic(Sigmoid), -1 ~ 1: hyperbolic tangent(Tanh)
-   loss함수
    -   MSELoss
-   평가지표
    -   MSE, RMSE, R square($R^2$)

## 다중분류 모델

-   output layer의 unit 개수
    -   정답 class(고유값)의 개수
-   출력 Layer에 적용하는 activation 함수
    -   Softmax: 클래스별 확률을 출력
-   loss함수
    -   **categrocial crossentropy**
    -   파이토치 함수
        -   **CrossEntropyLoss** = NLLLoss(정답) + LogSoftmax(모델 예측값)
        -   **NLLLoss**
            -   정답을 OneHot Encoding 처리 후 Loss를 계산한다.
            -   입력으로 LogSoftmax 처리한 모델 예측값과 onehot encoding 안 된 정답을 받는다.
        -   **LogSoftmax**
            -   입력값에 Softmax 계산후 그 Log를 계산한다.
                -   NLLLoss의 모델 예측값 입력값으로 처리할 때 사용한다.

```python
pred = model(input)
loss1 = nn.NLLLoss(nn.LogSoftmax(dim=-1)(pred), y)
# or
loss2 = nn.CrossEntropyLoss()(pred, y)
```

## 이진분류 모델

-   output layer의 unit 개수
    -   1개 (positive일 확률)
-   출력 Layer에 적용하는 activation 함수
    -   Sigmoid(Logistic)
-   loss 함수
    -   **Binary crossentropy**
    -   파이토치 함수: **BCELoss**
